# Song Recommendator Using APIs and WebScraping

![](https://r2.erweima.ai/i/QumLG5iITDShLUzIOP9Mqg.png)

## Let's start with the function of Billboard: HOT SONGS 🔥



In [1]:
# def scrape_billboard():
def billboard():
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    url = "https://www.billboard.com/charts/hot-100/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    songs=[]
    artists=[]
    lw_rank=[]
    weeks_on_chart=[]
    for song in soup.select("li h3", class_ = "c-title  a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 u-font-size-23@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-245 u-max-width-230@tablet-only u-letter-spacing-0028@tablet")[:100]:
        songs.append(song.get_text().strip())
    for artist in soup.select("li ul li span", class_= "c-label  a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet")[::7]:
        artists.append(artist.get_text().strip())
    for i in soup.select("div ul li ul li span", class_= "c-label  a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max lrv-u-padding-tb-050@mobile-max u-font-size-32@tablet")[1::7]:
        lw_rank.append(i.get_text().strip())
    for i in soup.select("div ul li ul li span", class_= "c-label  a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max lrv-u-padding-tb-050@mobile-max u-font-size-32@tablet")[3::7]:
        weeks_on_chart.append(i.get_text().strip())
    df = pd.DataFrame({"Song" : songs, "Artist" : artists, "Last_week_rank": lw_rank, "Weeks_on_chart": weeks_on_chart})
    return df


In [2]:
billboard()

,Song,Artist,Last_week_rank,Weeks_on_chart
0,4X4,Travis Scott,-,1
1,Die With A Smile,Lady Gaga & Bruno Mars,1,24
2,APT.,ROSE & Bruno Mars,3,15
3,Luther,Kendrick Lamar & SZA,5,10
4,A Bar Song (Tipsy),Shaboozey,4,42
...,...,...,...,...
95,Khe?,Rauw Alejandro & Romeo Santos,86,8
96,Me Jalo,Fuerza Regida & Grupo Frontera,-,1
97,Wondering Why,The Red Clay Strays,100,20
98,Boys Back Home,Dylan Marlowe & Dylan Scott,91,5


In [3]:
from billboard import billboard100 as bl
bl()

,Song,Artist,Last_week_rank,Weeks_on_chart
0,4X4,Travis Scott,-,1
1,Die With A Smile,Lady Gaga & Bruno Mars,1,24
2,APT.,ROSE & Bruno Mars,3,15
3,Luther,Kendrick Lamar & SZA,5,10
4,A Bar Song (Tipsy),Shaboozey,4,42
...,...,...,...,...
95,Khe?,Rauw Alejandro & Romeo Santos,86,8
96,Me Jalo,Fuerza Regida & Grupo Frontera,-,1
97,Wondering Why,The Red Clay Strays,100,20
98,Boys Back Home,Dylan Marlowe & Dylan Scott,91,5


# Let's apply some Python Logic 🐍

In [4]:
# def hot_recommender():
    # import scraping function & random int generator
def hot_recommender():
    from billboard import billboard100 as bl
    from random import randint
    # importamos el df de billboard y lo guardamos dentro de la variable
    billboard = bl()

    # pedir canción
    song= input("What song do you like?").strip().lower()
   # pasar todas las canciones de billboard + usuario a minusculas para poder compararlas
    billboard["song_lower"]= billboard["Song"].str.lower()

    # hacer un subset de nuestro df que tenga todas las opciones de la canción que ha introducido el usuario
    song_row = billboard[billboard["song_lower"].str.contains(song)]
    if len(song_row) == 0:
        print("Your song is not hot", flush= True) # fuerza al sistema a ir sacando prints
        # introducir extra info después para usar otro tipo de recomendador
        return
    else: 
        for i in range(len(song_row)):
            check_song=input("Did you mean " + song_row["Song"].values[i] + " by " + song_row["Artist"].values[i] + "?")
            if check_song.lower() in ["yes", "y", "si", "sí"]:
                print("That's a HOT SONG 🔥")
                song_selected = song_row["Song"].values[i]
                while True:
                    random_song=randint(0, len(billboard)-1)
                    if billboard["Song"][random_song]!= song_selected:
                        print(f'You might also like {billboard["Song"][random_song]} by {billboard["Artist"][random_song]}')
                        # abrir esa canción en spotify
                        break
                return 

        print("Not the song I had in mind... Let me check somewhere else!")
        # Introduciremos extra info DESPUÉS para usar otro tipo de recomendador

In [ ]:
hot_recommender() # me recomienda una canción dentro de billboard si la que le digo es hot, si no es hot no hace nada


That's a HOT SONG 🔥
You might also like No One Noticed by The Marias


## Third Step: Creation of a spotipy link to the recommended song to open the song 🎧

In [9]:
# def spoti_open(song_name): song name + artist name, lo vamos a pasar como query en spotify
# nos quedamos con el external url para poder abrirlo en el navegador
def spoti_open(song_name):
    from dotenv import load_dotenv
    import os

    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials

    import time
    import webbrowser

    load_dotenv()
    user=os.getenv("client")
    password=os.getenv("secret")
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=user, client_secret=password))

    opensong=sp.search(q=song_name, limit=1)
    browser=opensong["tracks"]["items"][0]["external_urls"]["spotify"]
    time.sleep(2)
    webbrowser.open(browser)

In [14]:
# Let's union la función HOT RECOMMENDER con SPOTI OPEN
def hot_recommender():
    # import scraping function & random int generator
    from billboard import billboard100 as bl
    from random import randint
    from spoti import spoti_open
    
    # scrape billboard hot 100
    billboard = bl()
    
    #Pedir canción y pasar todas las canciones de billboard + la del usuario a minúsculas para poder compararlas: 
    song = input("What song do you like? ").strip().lower()
    # check if song is on billboard hot 100
    billboard["song_lower"]=billboard["Song"].str.lower()

    #hacer un subset de nuestro df que tenga todas las opciones de la song que ha introducido el usuario. 
    song_row = billboard[billboard["song_lower"].str.contains(song)]
    if len(song_row) == 0:
        print("Your song is not hot", flush=True)
        #spoti(song)
        return
    else:
        for i in range(len(song_row)):
            check_song = input("Did you mean " + song_row["Song"].values[i] + " by " + song_row["Artist"].values[i] + "? ")  
        # if song is in billboard hot 100, recommend another random hot song
            if check_song.lower() in ["yes", "y", "si", "sí"]:
                print(song_row["Song"].values[i] + " by " + song_row["Artist"].values[i] + " " + "is a hot song.", flush=True)
                song_selected=song_row["Song"].values[i]
                while True:
                    random_song = randint(0, len(billboard)-1)
                    if billboard["Song"][random_song]!=song_selected:
                        print(f'You might also like " {billboard["Song"][random_song]} by {billboard["Artist"][random_song]}')
                        final_song=billboard["Song"][random_song]
                        spoti_open(final_song)
                        # abrir esa canción en spotify
                        break
                return
        
        print("Not the song I had in mind... Let me check somewhere else!")
        # Introduciremos extra info DESPUÉS para usar otro tipo de recomendador



In [15]:
hot_recommender()

Think I'm In Love With You by Chris Stapleton is a hot song.
You might also like " Tweaker by GELO


## Fourth Step: What if the song is not Hot? ❄️
## WE WANT A RECOMMENDATION 😡

In [16]:
def spoti_recommend(song):
    from dotenv import load_dotenv
    import os

    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials

    import time
    import webbrowser

    load_dotenv()
    user=os.getenv("client")
    password=os.getenv("secret")
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=user, client_secret=password))

    results=sp.search(q=song, limit=50)
    for i in results["tracks"]["items"]:
            isyoursong=input(f'Is your song {i["name"]} by {i["artists"][0]["name"]}')
            if isyoursong.lower() in ["yes", "y", "si", "sí"]:  # es esta tu canción? o esta? o esta? (hasta que diga que sí)
                songproposed=i["name"]
                artistproposed=i["artists"][0]["name"]
                artistid=i["artists"][0]["id"]
    #una vez tengamos la canción y el artista correcto, haremos una query para buscar específicamente una recomendación que no sea del mismo artista. 
                recommendations=sp.search(q=f'{songproposed+artistproposed}', limit=50)
                for j in recommendations["tracks"]["items"]:
                    if j["artists"][0]["id"]!=artistid:
                        print(f'I recommend you {j["name"]} by {j["artists"][0]["name"]}')
                        time.sleep(3)

                        browser=j["external_urls"]["spotify"]
                        webbrowser.open(browser)
                        return
    print("I am never gonna give you up... but we did not find a recommendation for your song. Good luck next time!")
    time.sleep(2)
    webbrowser.open("https://open.spotify.com/track/4PTG3Z6ehGkBFwjybzWkR8")
    return
    

In [17]:
spoti_recommend("goodbye")

I recommend you Kammerflimmern by RAUM27


## Now it is time to set it all together and split the functions in .PY DOCS! 🏢

In [19]:
# Your Final code!
def hot_recommender():
    # import scraping function & random int generator
    from billboard import billboard100 as bl
    from random import randint
    from spoti import spoti_open
    from spoti import spoti_recommend
    
    # scrape billboard hot 100
    billboard = bl()
    
    #Pedir canción y pasar todas las canciones de billboard + la del usuario a minúsculas para poder compararlas: 
    song = input("What song do you like? ").strip().lower()
    # check if song is on billboard hot 100
    billboard["song_lower"]=billboard["Song"].str.lower()

    #hacer un subset de nuestro df que tenga todas las opciones de la song que ha introducido el usuario. 
    song_row = billboard[billboard["song_lower"].str.contains(song)]
    if len(song_row) == 0:
        print("Your song is not hot", flush=True)
        spoti_recommend(song)
        return
    else:
        for i in range(len(song_row)):
            check_song = input("Did you mean " + song_row["Song"].values[i] + " by " + song_row["Artist"].values[i] + "? ")  
        # if song is in billboard hot 100, recommend another random hot song
            if check_song.lower() in ["yes", "y", "si", "sí"]:
                print(song_row["Song"].values[i] + " by " + song_row["Artist"].values[i] + " " + "is a HOT SONG 🔥", flush=True)
                song_selected=song_row["Song"].values[i]
                while True:
                    random_song = randint(0, len(billboard)-1)
                    if billboard["Song"][random_song]!=song_selected:
                        print(f'You might also like " {billboard["Song"][random_song]} by {billboard["Artist"][random_song]}')
                        final_song=billboard["Song"][random_song]
                        spoti_open(final_song)
                        # abrir esa canción en spotify
                        break
                return
        
        print("Not the song I had in mind... Let me check somewhere else!")
        spoti_recommend(song)


In [22]:
hot_recommender()

Your song is not hot
I recommend you +57 by KAROL G
